<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_expConv2Dv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

Everst_data=pd.read_csv('average_premium.csv')
data=Everst_data
data['Attained_age']=data['Age']+data['Duration']
data['Premium_rate']=data['BasePremiumAmount']/data['BaseNARAmount']*10000
#exclude_cols = ['Gender','RiskClass']
Coulmns_to_keep=['Age','Premium_rate','Gender','RiskClass','Duration']
#data=data[data['SingleOrJointType']=="Single"]
#data=data[data['Gender']=="Male"]
#data=data[data['RiskClass']=="NS1"]
data=data[Coulmns_to_keep]
data=data[(data['RiskClass']!="SS1") & (data['RiskClass']!="SS2")& (data['RiskClass']!="SP1")]
data['index']=data.index
numerical_columns = data.select_dtypes(include=['number']).columns
categorical_columns = data.select_dtypes(include=['object']).columns
data_copy=data.copy()
data

<ipython-input-1-c0094c1f61dc>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=data.index


,Age,Premium_rate,Gender,RiskClass,Duration,index
0,52.0,32.884713,Female,NP1,8.0,0
1,25.0,7.822222,Male,NP1,7.0,1
2,20.0,3.640212,Female,NS2,8.0,2
4,38.0,11.758730,Male,NP1,8.0,4
5,30.0,5.480635,Male,NS2,8.0,5
...,...,...,...,...,...,...
299968,39.0,3.321123,Female,NP1,5.0,299968
299969,23.0,4.979894,Male,NS2,5.0,299969
299970,53.0,59.015873,Female,NP1,2.0,299970
299971,37.0,16.080000,Female,NP1,7.0,299971


In [2]:
# Create a multi-index dataframe using Age, Duration, and a combined Gender and RiskClass column

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
data['RiskClass'] = data['RiskClass'].replace({'N+1': 'NP2', 'NP1': 'NP2'}).replace({'NS3': 'NS2', 'NS2': 'NS2', 'NR1': 'NS2', 'NS1' : 'NS2'})
print(data)
print(data['RiskClass'].unique())
multi_index_df = data.set_index(['Age', 'Duration', data['Gender'] + '_' + data['RiskClass']])

# Convert the multi-index dataframe to a regular dataframe
result_df = multi_index_df.reset_index().drop(['Gender', 'RiskClass'], axis=1)
print(result_df)
result_df['level_2'].unique()

         Age  Premium_rate  Gender RiskClass  Duration   index
0       52.0     32.884713  Female       NP2       8.0       0
1       25.0      7.822222    Male       NP2       7.0       1
2       20.0      3.640212  Female       NS2       8.0       2
4       38.0     11.758730    Male       NP2       8.0       4
5       30.0      5.480635    Male       NS2       8.0       5
...      ...           ...     ...       ...       ...     ...
299968  39.0      3.321123  Female       NP2       5.0  299968
299969  23.0      4.979894    Male       NS2       5.0  299969
299970  53.0     59.015873  Female       NP2       2.0  299970
299971  37.0     16.080000  Female       NP2       7.0  299971
299972   NaN           NaN     NaN       NaN       NaN  299972

[274271 rows x 6 columns]
['NP2' 'NS2' nan]
         Age  Duration     level_2  Premium_rate   index
0       52.0       8.0  Female_NP2     32.884713       0
1       25.0       7.0    Male_NP2      7.822222       1
2       20.0       8.0  Fema

array(['Female_NP2', 'Male_NP2', 'Female_NS2', 'Male_NS2', nan],
      dtype=object)

In [6]:
import numpy as np
import pandas as pd

# Before removing NAs
print("Before removing NAs:")
print(result_df.describe(include='all'))
print(result_df.count())

# Drop rows with missing values
result_df_no_na = result_df.dropna()

# After removing NAs
print("\nAfter removing NAs:")
print(result_df_no_na.describe(include='all'))
print(result_df_no_na.count())


Before removing NAs:
                  Age       Duration     level_2   Premium_rate          index
count   274270.000000  274270.000000      274270  274270.000000  274271.000000
unique            NaN            NaN           4            NaN            NaN
top               NaN            NaN  Female_NP2            NaN            NaN
freq              NaN            NaN       84222            NaN            NaN
mean        37.344081       4.792985         NaN      12.984724  149999.942119
std         11.193864       2.489008         NaN      22.221257   86598.955429
min         18.000000       1.000000         NaN     -41.844444       0.000000
25%         28.000000       3.000000         NaN       4.136054   74993.500000
50%         36.000000       4.000000         NaN       7.225397  149997.000000
75%         45.000000       7.000000         NaN      13.070899  224997.500000
max         70.000000      11.000000         NaN    1499.437037  299972.000000
Age             274270
Duration

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D,Conv2D, Flatten, Dense, MaxPooling1D,MaxPooling2D,Reshape,Conv2DTranspose

# Set a random seed for reproducibility
np.random.seed(42)

# Generate normal data
n_samples = 1000
ages = np.random.randint(18, 70, size=n_samples)
durations = np.random.randint(1, 20, size=n_samples)
premiums = ages * durations + np.random.normal(0, 50, size=n_samples)

# Introduce anomalies
n_anomalies = 50
anomaly_indices = np.random.choice(n_samples, n_anomalies, replace=False)
premiums[anomaly_indices] = premiums[anomaly_indices] + np.random.randint(500, 1000, size=n_anomalies)

# Create a DataFrame
data = pd.DataFrame({
    'age': ages,
    'duration': durations,
    'premium': premiums
})
#

In [5]:
# prompt: for the dsata  result_df_no_na one hot encode the level_2

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assuming 'result_df_no_na' is your DataFrame from the preceding code

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the 'level_2' column and transform it
encoded_data = encoder.fit_transform(result_df_no_na[['level_2']])

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['level_2']))

# Concatenate the encoded DataFrame with the original DataFrame
result_df_encoded = pd.concat([result_df_no_na, encoded_df], axis=1)

# Optionally, drop the original 'level_2' column
result_df_encoded = result_df_encoded.drop('level_2', axis=1)

result_df_encoded.drop(['index'], axis=1,inplace=True)
result_df_encoded

,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2
0,52.0,8.0,32.884713,1.0,0.0,0.0,0.0
1,25.0,7.0,7.822222,0.0,0.0,1.0,0.0
2,20.0,8.0,3.640212,0.0,1.0,0.0,0.0
3,38.0,8.0,11.758730,0.0,0.0,1.0,0.0
4,30.0,8.0,5.480635,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
274265,21.0,2.0,2.679365,0.0,0.0,1.0,0.0
274266,39.0,5.0,3.321123,1.0,0.0,0.0,0.0
274267,23.0,5.0,4.979894,0.0,0.0,0.0,1.0
274268,53.0,2.0,59.015873,1.0,0.0,0.0,0.0


In [12]:
# prompt: from result_df_no_na only keep those records where for each age and duration of theat record there are records for level_2 from each of the unique values of level_2

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D,Conv2D, Flatten, Dense, MaxPooling1D,MaxPooling2D,Reshape,Conv2DTranspose
from sklearn.preprocessing import OneHotEncoder

# Assuming 'result_df_no_na' is your DataFrame
# Get unique values of 'level_2'
unique_levels = result_df_no_na['level_2'].unique()

# Group the DataFrame by 'Age' and 'Duration'
grouped = result_df_no_na.groupby(['Age', 'Duration'])

# Filter groups that have all unique levels
filtered_df = grouped.filter(lambda x: set(x['level_2']) == set(unique_levels))

filtered_df


,Age,Duration,level_2,Premium_rate,index
0,52.0,8.0,Female_NP2,32.884713,0
1,25.0,7.0,Male_NP2,7.822222,1
2,20.0,8.0,Female_NS2,3.640212,2
3,38.0,8.0,Male_NP2,11.758730,4
4,30.0,8.0,Male_NS2,5.480635,5
...,...,...,...,...,...
274265,21.0,2.0,Male_NP2,2.679365,299967
274266,39.0,5.0,Female_NP2,3.321123,299968
274267,23.0,5.0,Male_NS2,4.979894,299969
274268,53.0,2.0,Female_NP2,59.015873,299970


In [26]:
# prompt: add e ncoding on level2 in filtrerd df also order teh ata by age an duration so that only similar data for age and duration are together in the next steps

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D,Conv2D, Flatten, Dense, MaxPooling1D,MaxPooling2D,Reshape,Conv2DTranspose
from sklearn.preprocessing import OneHotEncoder

# Assuming 'filtered_df' is your DataFrame from the preceding code

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the 'level_2' column and transform it
encoded_data = encoder.fit_transform(filtered_df[['level_2']])

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['level_2']))

# Concatenate the encoded DataFrame with the original DataFrame
filtered_df_encoded = pd.concat([filtered_df, encoded_df], axis=1)

# Optionally, drop the original 'level_2' column
filtered_df_encoded = filtered_df_encoded.drop('level_2', axis=1)

# Order the data by 'Age' and 'Duration'
filtered_df_encoded = filtered_df_encoded.sort_values(by=['Age', 'Duration'])

filtered_df_encoded


,Age,Duration,Premium_rate,index,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2
235,18.0,1.0,2.041270,256.0,0.0,0.0,0.0,1.0
252,18.0,1.0,9.630476,276.0,0.0,0.0,0.0,1.0
1243,18.0,1.0,7.958730,1358.0,1.0,0.0,0.0,0.0
1245,18.0,1.0,1.950265,1360.0,1.0,0.0,0.0,0.0
2365,18.0,1.0,2.300529,2568.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
262442,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
262498,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
262500,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
262508,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0


In [27]:
import numpy as np

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Get unique values of 'level_2' (assuming it's still available in the original DataFrame)
level_2_values = filtered_df['level_2'].unique()

# Determine the maximum number of samples across all level_2 groups
max_samples = 0
for level_2 in level_2_values:
    filtered_df2 = filtered_df[filtered_df_encoded['level_2_' + level_2] == 1]
    max_samples = max(max_samples, filtered_df.shape[0])

# Initialize an empty list to store the layers
layers = []
for level_2 in level_2_values:
    # Filter the DataFrame for the current 'level_2' value
    filtered_df2 = filtered_df_encoded[(filtered_df_encoded['level_2_' + level_2] == 1) ]   # Extract the relevant columns (Age, Duration, Premium_rate, Premium_Diff)
    layer_data = filtered_df2[['Age', 'Duration', 'Premium_rate']].values

    # Pad the layer data with zeros to match the maximum number of samples
    padded_layer_data = np.pad(layer_data, ((0, max_samples - layer_data.shape[0]), (0, 0)), 'constant')

    # Append the padded layer data to the list
    layers.append(padded_layer_data)

# Stack the layers into a 4D array
stacked_array = np.stack(layers, axis=-1)

stacked_array # Output: (max number of samples, 3, number of level_2 values)


<ipython-input-27-31868c09321a>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df2 = filtered_df[filtered_df_encoded['level_2_' + level_2] == 1]
<ipython-input-27-31868c09321a>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df2 = filtered_df[filtered_df_encoded['level_2_' + level_2] == 1]
<ipython-input-27-31868c09321a>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df2 = filtered_df[filtered_df_encoded['level_2_' + level_2] == 1]
<ipython-input-27-31868c09321a>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df2 = filtered_df[filtered_df_encoded['level_2_' + level_2] == 1]


array([[[18.        , 18.        , 18.        , 18.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ],
        [ 7.95873016,  2.09015873,  2.3005291 ,  2.04126984]],

       [[18.        , 18.        , 18.        , 18.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ],
        [ 1.95026455,  9.62962963,  9.62962963,  9.63047619]],

       [[18.        , 18.        , 18.        , 18.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ],
        [ 2.09100529,  2.3015873 ,  2.29982363,  9.62651727]],

       ...,

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,

In [28]:
#test_df_with_anomalies = result_df_encoded.iloc[X_test.index].copy()  # X_test is a numpy array and does not have an index
# Reshape the data to 3D for CNN input
#data_array = result_df_encoded.values  # Extract the underlying NumPy array
data_reshaped = stacked_array.reshape(-1, 1, 3, 4)
data_reshaped

array([[[[18.        , 18.        , 18.        , 18.        ],
         [ 1.        ,  1.        ,  1.        ,  1.        ],
         [ 7.95873016,  2.09015873,  2.3005291 ,  2.04126984]]],


       [[[18.        , 18.        , 18.        , 18.        ],
         [ 1.        ,  1.        ,  1.        ,  1.        ],
         [ 1.95026455,  9.62962963,  9.62962963,  9.63047619]]],


       [[[18.        , 18.        , 18.        , 18.        ],
         [ 1.        ,  1.        ,  1.        ,  1.        ],
         [ 2.09100529,  2.3015873 ,  2.29982363,  9.62651727]]],


       ...,


       [[[ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]]],


       [[[ 0.        

In [29]:
# prompt: drop all eleemts in the data_reshaped array where there are zero values in anyof the channels , i.e the last dimeion and prmt th etresult again

import numpy as np

# Find indices of arrays with zero values in any channel
zero_indices = np.any(data_reshaped == 0, axis=(1, 2, 3))

# Remove arrays with zero values
data_filtered = data_reshaped[~zero_indices]

print(data_filtered.shape)
data_filtered

(49597, 1, 3, 4)


array([[[[18.        , 18.        , 18.        , 18.        ],
         [ 1.        ,  1.        ,  1.        ,  1.        ],
         [ 7.95873016,  2.09015873,  2.3005291 ,  2.04126984]]],


       [[[18.        , 18.        , 18.        , 18.        ],
         [ 1.        ,  1.        ,  1.        ,  1.        ],
         [ 1.95026455,  9.62962963,  9.62962963,  9.63047619]]],


       [[[18.        , 18.        , 18.        , 18.        ],
         [ 1.        ,  1.        ,  1.        ,  1.        ],
         [ 2.09100529,  2.3015873 ,  2.29982363,  9.62651727]]],


       ...,


       [[[41.        ,         nan, 45.        , 52.        ],
         [ 2.        ,         nan,  9.        ,  8.        ],
         [ 6.68042328,         nan, 19.88888889, 32.9015873 ]]],


       [[[41.        ,         nan, 45.        , 52.        ],
         [ 2.        ,         nan,  9.        ,  8.        ],
         [18.47301587,         nan, 23.61955556, 94.06666667]]],


       [[[41.        

In [30]:
# prompt: drop any nan as ewll if its found

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D,Conv2D, Flatten, Dense, MaxPooling1D,MaxPooling2D,Reshape,Conv2DTranspose
from sklearn.preprocessing import OneHotEncoder

# ... (preceding code)

# Find indices of arrays with zero values or NaN values in any channel
invalid_indices = np.any(np.isnan(data_reshaped) | (data_reshaped == 0), axis=(1, 2, 3))

# Remove arrays with zero values or NaN values
data_filtered = data_reshaped[~invalid_indices]

print(data_filtered.shape)
data_filtered


(47511, 1, 3, 4)


array([[[[ 18.        ,  18.        ,  18.        ,  18.        ],
         [  1.        ,   1.        ,   1.        ,   1.        ],
         [  7.95873016,   2.09015873,   2.3005291 ,   2.04126984]]],


       [[[ 18.        ,  18.        ,  18.        ,  18.        ],
         [  1.        ,   1.        ,   1.        ,   1.        ],
         [  1.95026455,   9.62962963,   9.62962963,   9.63047619]]],


       [[[ 18.        ,  18.        ,  18.        ,  18.        ],
         [  1.        ,   1.        ,   1.        ,   1.        ],
         [  2.09100529,   2.3015873 ,   2.29982363,   9.62651727]]],


       ...,


       [[[ 40.        ,  70.        ,  44.        ,  50.        ],
         [  3.        ,   4.        ,   6.        ,   5.        ],
         [  5.40063492,  81.84126984,  14.52063492,  18.4       ]]],


       [[[ 40.        ,  70.        ,  44.        ,  50.        ],
         [  3.        ,   4.        ,   6.        ,   5.        ],
         [  6.44021164,  81.8412

In [37]:
X_train
X_train.shape

(38008, 1, 3, 4)

In [32]:

# Split the data into training and test sets
# Split the data into training and test sets
X_train, X_test, train_indices, test_indices = train_test_split(
    data_filtered,
    np.arange(data_filtered.shape[0]),  # Create indices for the array
    test_size=0.2,
    random_state=42
)
#train_indices, test_indices = train_test_split(data_reshaped.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)

conv_layer = Conv2D(32, (1,3), activation="relu",padding='same')(input_layer)
conv_layer = MaxPooling2D(pool_size=(1,1))(conv_layer)

output_shape = (conv_layer.shape[1], conv_layer.shape[2], conv_layer.shape[3])  # Adjust as needed

# Reshape the output of the convolutional layers
reshape_layer = Reshape(output_shape)(conv_layer)

# Use transposed convolution (Conv2DTranspose) to upsample and reconstruct
deconv_layer = Conv2DTranspose(32, (1, 3), activation='relu', padding='same')(reshape_layer)
output_layer = Conv2D(X_train.shape[3], (1, 1), activation='linear', padding='same')(deconv_layer)  # Output 4 channels

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# ... (rest of your code for training and prediction)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=50,
                        batch_size=32,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train - pred_train,2), axis=1)



Epoch 1/50
1069/1069 [==============================] - 5s 4ms/step - loss: 9.8263 - val_loss: 0.0941
Epoch 2/50
1069/1069 [==============================] - 3s 3ms/step - loss: 0.0574 - val_loss: 0.0533
Epoch 3/50
1069/1069 [==============================] - 3s 2ms/step - loss: 0.0393 - val_loss: 0.0343
Epoch 4/50
1069/1069 [==============================] - 2s 2ms/step - loss: 0.0257 - val_loss: 0.0298
Epoch 5/50
1069/1069 [==============================] - 4s 3ms/step - loss: 0.0183 - val_loss: 0.0236
Epoch 6/50
1069/1069 [==============================] - 4s 4ms/step - loss: 0.0235 - val_loss: 0.0212
Epoch 7/50
1069/1069 [==============================] - 3s 3ms/step - loss: 0.0148 - val_loss: 0.0177
Epoch 8/50
1069/1069 [==============================] - 3s 2ms/step - loss: 0.0078 - val_loss: 0.0179
Epoch 9/50
1069/1069 [==============================] - 2s 2ms/step - loss: 0.0133 - val_loss: 0.0171
Epoch 10/50
1069/1069 [==============================] - 3s 3ms/step - loss: 0.006

In [33]:
pred_train

array([[[[31.972307  , 40.00123   , 34.051006  , 36.973846  ],
         [ 2.9870317 ,  1.9752192 ,  3.9910448 ,  2.9981515 ],
         [ 3.3882978 ,  3.9726245 ,  6.6289487 ,  5.53451   ]]],


       [[[28.005468  , 32.9821    , 28.97913   , 31.020714  ],
         [ 3.9718254 ,  2.9896245 ,  8.000642  ,  5.0119658 ],
         [ 6.0242825 ,  5.380406  ,  5.4330883 ,  6.232376  ]]],


       [[[26.98937   , 31.993385  , 29.000933  , 30.012524  ],
         [ 6.979805  ,  1.973206  ,  1.0003712 ,  5.005399  ],
         [ 4.425792  ,  2.1919494 ,  4.79101   ,  4.2905316 ]]],


       ...,


       [[[36.01559   , 46.983826  , 38.99002   , 43.00821   ],
         [ 2.9846003 ,  6.984909  ,  4.0057    ,  2.0082014 ],
         [ 4.854017  , 20.876736  ,  4.537457  ,  5.2485304 ]]],


       [[[19.024096  , 19.006693  , 19.014526  , 18.988003  ],
         [ 0.99281967,  5.9947777 ,  3.0060856 ,  3.9952428 ],
         [ 2.2949164 , 11.452527  ,  2.9816148 ,  9.600874  ]]],


       [[[26.972803  

In [34]:

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test - pred_test  , 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 95)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

X_test
pred_test

297/297 [==============================] - 1s 2ms/step
Threshold: 0.001513257753003927
Number of anomalies in the test set: 5586


array([[[[30.006018  , 35.983788  , 31.986225  , 34.01788   ],
         [ 3.9814904 ,  6.9824476 ,  3.0026877 ,  4.0104017 ],
         [ 3.880396  ,  8.350938  ,  2.6971538 ,  4.4835405 ]]],


       [[[23.984531  , 26.979704  , 24.985565  , 26.032402  ],
         [ 7.9688425 ,  0.9660402 ,  5.049216  ,  1.9527314 ],
         [ 8.715238  ,  1.8196716 ,  3.1029289 ,  1.7817905 ]]],


       [[[37.038433  , 50.98192   , 40.95598   , 45.025665  ],
         [ 5.9813952 ,  3.9833617 ,  3.0017512 ,  5.0155087 ],
         [ 7.8668957 , 22.75681   ,  6.08524   , 13.917496  ]]],


       ...,


       [[[22.013504  , 24.004747  , 22.005016  , 22.99803   ],
         [ 0.9931549 ,  0.9856997 ,  5.9866147 ,  2.985222  ],
         [ 1.9236456 ,  1.6129543 ,  8.497366  ,  6.724445  ]]],


       [[[33.002647  , 42.989616  , 36.005573  , 38.995117  ],
         [ 8.987646  ,  3.9812865 ,  5.9868164 ,  8.006377  ],
         [ 8.048571  ,  8.821319  ,  7.307101  , 12.701252  ]]],


       [[[30.002256  

In [36]:
# Create a copy of the original test DataFrame
test_df_with_anomalies = filtered_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies['anomaly'] = anomalies

print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies

Threshold: 0.001513257753003927
Number of anomalies in the test set: 5586


ValueError: Shape of passed values is (9, 6, 4), indices imply (10, 9)

ValueError: Shape of passed values is (9, 6, 4), indices imply (10, 9)

In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 455


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
21185,22.0,4.0,8.359788,0.0,0.0,1.0,0.0,False
21081,41.0,2.0,4.408889,1.0,0.0,0.0,0.0,False
38072,35.0,9.0,9.813333,0.0,0.0,0.0,1.0,False
37127,21.0,9.0,7.733333,0.0,0.0,0.0,1.0,False
22786,38.0,3.0,5.911330,0.0,0.0,0.0,1.0,False
...,...,...,...,...,...,...,...,...
25543,42.0,3.0,8.059259,0.0,0.0,1.0,0.0,False
23315,28.0,5.0,7.714286,0.0,0.0,1.0,0.0,False
8920,41.0,7.0,12.370370,0.0,0.0,1.0,0.0,False
21822,47.0,5.0,15.828571,0.0,0.0,1.0,0.0,False
